In [28]:
import json

def overlap(a, b):
    a_left = a[0]
    a_right = a[1]
    b_left = b[0]
    b_right = b[1]
    if (a_left >= b_left and a_left <= b_right) \
        or (a_right >= b_left and a_right <= b_right):
        return 1
    return 0

def equal(a, b):
    if a[0] == b[0] and a[1] == b[1]:
        return 1
    return 0

def write_to_file(filename, data, result):
    dict = {
        "data": data,
        "result": result
    }
    fw = open("result/" + filename, "a", encoding="utf8")
    fw.writelines(json.dumps(dict, ensure_ascii=False))
    fw.writelines("\n")

def result_analysis():
    cor = [0, 0]
    inc = [0, 0]
    inc_relation = [0, 0]
    inc_s = [0, 0]
    inc_o = [0, 0]
    inc_spans = [0, 0]
    tot = [0, 0]

    with open("res.json", encoding='utf8') as fr:
        pred = []
        for line in fr:
            pred.append(json.loads(line))
    with open("data/bdci/train_bdci.json", encoding='utf8') as fr:
        data = []
        for line in fr:
            data.append(json.loads(line))

    for i in range(0, 1491):
        idx_str = 'AT' + str(i + 1).rjust(4, '0')
        sample_pred_tmp = pred[i]["spo_list"]
        sample_data_tmp = data[i]["spo_list"]
        sample_pred = []
        sample_data = []

        for spo in sample_pred_tmp:
            s_list = [spo["h"]["pos"][0], spo["h"]["pos"][1], spo["h"]["name"]]
            o_list = [spo["t"]["pos"][0], spo["t"]["pos"][1], spo["t"]["name"]]
            p = spo["relation"]
            dict = {
                "s": s_list,
                "p": p,
                "o": o_list
            }
            sample_pred.append(dict)

        for spo in sample_data_tmp:
            s_list = [spo["h"]["pos"][0], spo["h"]["pos"][1], spo["h"]["name"]]
            o_list = [spo["t"]["pos"][0], spo["t"]["pos"][1], spo["t"]["name"]]
            p = spo["relation"]
            dict = {
                "s": s_list,
                "p": p,
                "o": o_list
            }
            sample_data.append(dict)
        
        for pred_rel in sample_pred:
            exists_cor = 0
            tot[0] += 1
            for data_rel in sample_data:
                if pred_rel["s"] == data_rel["s"] and pred_rel["p"] == data_rel["p"] \
                    and pred_rel["o"] == data_rel["o"]:
                    cor[0] += 1
                    exists_cor = 1
            if exists_cor == 0:
                inc[0] += 1
                for data_rel in sample_data:
                    if overlap(pred_rel["s"], data_rel["s"]) and overlap(pred_rel["o"], data_rel["o"]):
                        inc_spans[0] += 1
                    if overlap(pred_rel["s"], data_rel["s"]) and overlap(pred_rel["o"], data_rel["o"]) \
                        and pred_rel["p"] != data_rel["p"]:
                        inc_relation[0] += 1
                    if pred_rel["p"] == data_rel["p"] and overlap(pred_rel["s"], data_rel["s"]) \
                        and overlap(pred_rel["o"], data_rel["o"]) == 0:
                        if equal(pred_rel["s"], data_rel["s"]) == 0:
                            inc_spans[0] += 1
                        inc_o[0] += 1
                    if pred_rel["p"] == data_rel["p"] and overlap(pred_rel["s"], data_rel["s"]) == 0 \
                        and overlap(pred_rel["o"], data_rel["o"]):
                        if equal(pred_rel["o"], data_rel["o"]) == 0:
                            inc_spans[0] += 1
                        inc_s[0] += 1
            
        for data_rel in sample_data:
            exists_cor = 0
            tot[1] += 1
            for pred_rel in sample_pred:
                if pred_rel["s"] == data_rel["s"] and pred_rel["p"] == data_rel["p"] \
                    and pred_rel["o"] == data_rel["o"]:
                    cor[1] += 1
                    exists_cor = 1
            if exists_cor == 0:
                inc[1] += 1
                for pred_rel in sample_pred:
                    if overlap(pred_rel["s"], data_rel["s"]) and overlap(pred_rel["o"], data_rel["o"]):
                        inc_spans[1] += 1
                        write_to_file("inc_spans.json", data_rel, pred_rel)
                    if overlap(pred_rel["s"], data_rel["s"]) and overlap(pred_rel["o"], data_rel["o"]) \
                        and pred_rel["p"] != data_rel["p"]:
                        inc_relation[1] += 1
                        write_to_file("inc_relation.json", data_rel, pred_rel)
                    if pred_rel["p"] == data_rel["p"] and overlap(pred_rel["s"], data_rel["s"]) \
                        and overlap(pred_rel["o"], data_rel["o"]) == 0:
                        if equal(pred_rel["s"], data_rel["s"]) == 0:
                            inc_spans[1] += 1
                        inc_o[1] += 1
                        write_to_file("inc_o.json", data_rel, pred_rel)
                    if pred_rel["p"] == data_rel["p"] and overlap(pred_rel["s"], data_rel["s"]) == 0 \
                        and overlap(pred_rel["o"], data_rel["o"]):
                        if equal(pred_rel["o"], data_rel["o"]) == 0:
                            inc_spans[1] += 1
                        inc_s[1] += 1
                        write_to_file("inc_s.json", data_rel, pred_rel)
    
    print("for all predicted relations")
    print(f'#total: {tot[0]}')
    print(f'#correct: {cor[0]}')
    print(f'#incorrect: {inc[0]}')
    print(f'#incorrect spans: {inc_spans[0]}')
    print(f'#incorrect relation: {inc_relation[0]}')
    print(f'#incorrect subject: {inc_s[0]}')
    print(f'#incorrect object: {inc_o[0]}')
    print()

    print("for all true relations")
    print(f'#total: {tot[1]}')
    print(f'#correct: {cor[1]}')
    print(f'#incorrect: {inc[1]}')
    print(f'#incorrect spans: {inc_spans[1]}')
    print(f'#incorrect relation: {inc_relation[1]}')
    print(f'#incorrect subject: {inc_s[1]}')
    print(f'#incorrect object: {inc_o[1]}')

result_analysis()

for all predicted relations
#total: 6159
#correct: 5666
#incorrect: 500
#incorrect spans: 225
#incorrect relation: 4
#incorrect subject: 94
#incorrect object: 87

for all true relations
#total: 6701
#correct: 5666
#incorrect: 1035
#incorrect spans: 150
#incorrect relation: 2
#incorrect subject: 80
#incorrect object: 82


In [24]:
with open("res.json", encoding='utf8') as fr:
    pred = []
    for line in fr:
        pred.append(json.loads(line))
with open("data/bdci/train_bdci.json", encoding='utf8') as fr:
    true = []
    for line in fr:
        true.append(json.loads(line))

In [21]:
def correct_relation(rel1, rel2):
    if rel1['h']['pos'] != rel2['h']['pos']:
        return 0
    if rel1['t']['pos'] != rel2['t']['pos']:
        return 0
    if rel1['relation'] != rel2['relation']:
        return 0
    return 1

def get_vals(pred_list, true_list):
    TP, FP, FN = 0, 0, 0
    for pred_rel in pred_list:
        for true_rel in true_list:
            if correct_relation(pred_rel, true_rel):
                TP += 1
                break
    FP = len(pred_list) - TP
    FN = len(true_list) - TP
    return TP, FP, FN

def get_f1(pred_lists, true_lists):
    TP, FP, FN = 0, 0, 0
    if len(pred_lists) != len(true_lists):
        print('length of two lists are different')
        return
    for i in range(len(pred_lists)):
        tp, fp, fn = get_vals(pred_lists[i], true_lists[i])
        TP += tp
        FP += fp
        FN += fn
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    print(f'precision: {precision}')
    print(f'recall: {recall}')
    print(f'f1 score: {f1}')
    return precision, recall, f1

In [27]:
# calculate f1 scores according to length of text

text_len = [[] for i in range(10)]

for i, sample in enumerate(pred):
    idx = (int)(len(sample['text']) / 100)
    if idx >= 10:
        idx = 9
    text_len[idx].append(i)

for i in range(10):
    idx_list = text_len[i]
    pred_lists, true_lists = [], []
    for idx in idx_list:
        pred_lists.append(pred[idx]['spo_list'])
        true_lists.append(true[idx]['spo_list'])
    print(f'for sentence length {i*100} to {(i+1)*100}:')
    get_f1(pred_lists, true_lists)
    print()

for sentence length 0 to 100:
precision: 0.9718670076726342
recall: 0.8501118568232662
f1 score: 0.9069212410501194

for sentence length 100 to 200:
precision: 0.9661631419939577
recall: 0.8035175879396985
f1 score: 0.8773662551440329

for sentence length 200 to 300:
precision: 0.9315164220824598
recall: 0.9167812929848693
f1 score: 0.9240901213171577

for sentence length 300 to 400:
precision: 0.8791946308724832
recall: 0.9003436426116839
f1 score: 0.8896434634974533

for sentence length 400 to 500:
precision: 0.8602739726027397
recall: 0.839572192513369
f1 score: 0.8497970230040596

for sentence length 500 to 600:
precision: 0.8125
recall: 0.7857142857142857
f1 score: 0.7988826815642458

for sentence length 600 to 700:
precision: 0.7484662576687117
recall: 0.648936170212766
f1 score: 0.6951566951566952

for sentence length 700 to 800:
precision: 0.7592592592592593
recall: 0.8913043478260869
f1 score: 0.8200000000000001

for sentence length 800 to 900:
precision: 0.5714285714285714
re

In [25]:
# calculate f1 scores according to relation type

true_rel_type = [[] for i in range(4)]
pred_rel_type = [[] for i in range(4)]  # 1:部件故障 2:性能故障 3:检测工具 4:组成
rel2id = {
    '部件故障': 0,
    '性能故障': 1,
    '检测工具': 2,
    '组成': 3
}

for i, sample in enumerate(pred):
    pred_tmp_list = [[] for _ in range(4)]
    for rel in sample['spo_list']:
        pred_tmp_list[rel2id[rel['relation']]].append(rel)
    true_tmp_list = [[] for _ in range(4)]
    for rel in true[i]['spo_list']:
        true_tmp_list[rel2id[rel['relation']]].append(rel)
    for j in range(4):
        pred_rel_type[j].append(pred_tmp_list[j])
        true_rel_type[j].append(true_tmp_list[j])

for i in range(4):
    print(f'for rel type {i}:')
    get_f1(pred_rel_type[i], true_rel_type[i])
    print()

for rel type 0:
precision: 0.9225908372827805
recall: 0.8741061034425411
f1 score: 0.8976942783945346

for rel type 1:
precision: 0.8909512761020881
recall: 0.8687782805429864
f1 score: 0.8797250859106529

for rel type 2:
precision: 1.0
recall: 0.17857142857142858
f1 score: 0.30303030303030304

for rel type 3:
precision: 0.5384615384615384
recall: 0.06422018348623854
f1 score: 0.11475409836065575



In [26]:
pred_type_cnt = [0 for _ in range(4)]
true_type_cnt = [0 for _ in range(4)]

rel2id = {
    '部件故障': 0,
    '性能故障': 1,
    '检测工具': 2,
    '组成': 3
}

for i, sample in enumerate(pred):
    for spo in sample['spo_list']:
        pred_type_cnt[rel2id[spo['relation']]] += 1
    for spo in true[i]['spo_list']:
        true_type_cnt[rel2id[spo['relation']]] += 1

print('prediction:')
for i in range(4):
    print(f'{i}: {pred_type_cnt[i]}')
print()
print('true:')
for i in range(4):
    print(f'{i}: {true_type_cnt[i]}')

prediction:
0: 5697
1: 431
2: 5
3: 26

true:
0: 6013
1: 442
2: 28
3: 218
